# Notebook

In [1]:
import pandas as pd # pip install pandas
import numpy as np

# internal lib
import core as core

# Rendering
%matplotlib inline
import matplotlib # pip install matplotlib
import matplotlib.pyplot as plt

import plotly # pip install plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

import folium # pip install folium
from folium import plugins
from datetime import datetime

import ipywidgets as widgets

## Parameters

In [2]:
root = "../"
data_folder = root + "trip_data/"
fare_folder = root + "trip_fare/"
data_file = data_folder + "trip_data_1.csv"
fare_file = fare_folder + "trip_fare_1.csv"

nrows = None

## Loaded and cleaned

In [3]:
full = core.load_and_clean(data_file, fare_file, nrows=nrows)

The indexing is unique, the join operation is possible
Data loading and cleaning completed


In [4]:
full.describe()

,rate_code,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
count,1.477662e+07,1.477662e+07,1.477662e+07,1.477662e+07,1.477662e+07,1.477662e+07,1.477653e+07,1.477653e+07,1.477662e+07,1.477662e+07,1.477662e+07,1.477662e+07,1.477662e+07,1.477662e+07
mean,1.034273e+00,1.697372e+00,6.834236e+02,2.770976e+00,-7.263634e+01,4.001440e+01,-7.259443e+01,3.999219e+01,1.166472e+01,3.204904e-01,4.983970e-01,1.267509e+00,2.018670e-01,1.395298e+01
std,3.387715e-01,1.365396e+00,4.944063e+02,3.305923e+00,1.013819e+01,7.789904e+00,1.028860e+01,7.537067e+00,9.639219e+00,3.675741e-01,2.826541e-02,2.046084e+00,1.035481e+00,1.146469e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.771285e+03,-3.547921e+03,-2.350956e+03,-3.547921e+03,2.500000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e+00
25%,1.000000e+00,1.000000e+00,3.600000e+02,1.000000e+00,-7.399188e+01,4.073551e+01,-7.399121e+01,4.073468e+01,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,7.700000e+00
50%,1.000000e+00,1.000000e+00,5.540000e+02,1.700000e+00,-7.398166e+01,4.075315e+01,-7.398013e+01,4.075362e+01,9.000000e+00,0.000000e+00,5.000000e-01,8.000000e-01,0.000000e+00,1.050000e+01
75%,1.000000e+00,2.000000e+00,8.850000e+02,3.060000e+00,-7.396684e+01,4.076729e+01,-7.396390e+01,4.076819e+01,1.300000e+01,5.000000e-01,5.000000e-01,2.000000e+00,0.000000e+00,1.550000e+01
max,2.100000e+02,2.550000e+02,1.080000e+04,1.000000e+02,1.124042e+02,3.310365e+03,2.228738e+03,3.477106e+03,5.000000e+02,1.250000e+01,5.000000e-01,2.000000e+02,2.000000e+01,6.500000e+02


In [5]:
resample_columns = ["fare_amount", "surcharge", "mta_tax", "tip_amount", "tolls_amount", "total_amount"]
hour_sample = core.resample_on_subset(full, resample_columns, "H", np.sum)
minute_sample = core.resample_on_subset(full, resample_columns, "10T", np.sum)

In [6]:
selection = "total_amount" # choose something in the resample_columns list
core.plot_time_series(hour_sample[selection], minute_sample[selection])

In [26]:
# Warning, Chrome restriction to 66k point for the heatmap. The viz is limited to a few hours scope
date = pd.to_datetime("2013-01-10T09:10:00")
selection = "total_amount"#"tolls_amount"
df = full.loc[((full.index.get_level_values('pickup_datetime').day == date.day) & (full.index.get_level_values('pickup_datetime').hour == date.hour)) ]
core.heatmap(df, lat_col='pickup_latitude', lon_col='pickup_longitude', \
    zoom_start=11, heat_map_weights_col=selection, heat_map_radius=10)